In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import math
import decimal
from decimal import Decimal

In [ ]:
# ytm calculator
decimal.getcontext().rounding = "ROUND_HALF_UP"

current_bond_price = int(input('Current Bond Price: '))
bond_par_value = int(input('Bond Par Value: '))
rate = 0.01 * int(input('Bond Coupon Rate(%): '))
years = int(input('Years to Maturity: '))
payment = int(input('Payments a yaer: '))

nper = years * payment  # 期數
lst = [-current_bond_price] + [bond_par_value*rate/payment]*(nper-1) + [bond_par_value+bond_par_value*rate/payment]

ytm = Decimal(sp.irr(lst)*100).quantize(Decimal('0.0000'))

print('Yield to Maturity(YTM) = {}%'.format(ytm), sep ='')


In [ ]:
# 由 ytm 反推 Current Bond Price
num = 0
for i in range(1,11):
    num += lst[i]/(1+0.05376)**i
num = Decimal(num).quantize(Decimal('0'))
print(num)

In [ ]:
# spot rate calculator
decimal.getcontext().rounding = "ROUND_HALF_UP"

t = int(input('Duration of spot rate (years): '))
p = float(input('Price of {} year unit zero-coupon bond: '.format(t)))

y = p ** (-1/t) -1
Y = (-1/t) * math.log(p)

y = Decimal(100*y).quantize(Decimal('0.00'))
Y = Decimal(100*Y).quantize(Decimal('0.00'))

print('{} year spot rate of interest: '.format(t) + str(y) + '%')
print('{} year spot force of interest: '.format(t) + str(Y) + '%')

In [ ]:
# forward rate calculator
decimal.getcontext().rounding = "ROUND_HALF_UP"

t = int(input('Time due for the beginning of forward rate (years): '))
r = int(input('Duration of forward rate (years): '))
Pt = float(input('Price of {} year unit zero coupon bond: '.format(t)))
Pt_r = float(input('Price of {} year unit zero coupon bond: '.format(t+r)))

y = (Pt/Pt_r)**(1/r) - 1
Y = (1/r) * math.log(Pt/Pt_r)

y = Decimal(100*y).quantize(Decimal('0.00'))
Y = Decimal(100*Y).quantize(Decimal('0.00'))

print('{} year forward rate of interest beginning {} years from now: '.format(r,t) + str(y) + '%')
print('{} year forward force of interest beginning {} years from now: '.format(r,t) + str(Y) + '%')

In [3]:
decimal.getcontext().rounding = "ROUND_HALF_UP"

current_bond_price = int(input('Current Bond Price: '))
bond_par_value = int(input('Bond Par Value: '))
rate = 0.01 * int(input('Bond Coupon Rate(%): '))
years = int(input('Years to Maturity: '))
payment = int(input('Payments a yaer: '))

nper = years * payment  # 期數
lst_ = [-current_bond_price] + [bond_par_value*rate/payment]*(nper-1) + [bond_par_value+bond_par_value*rate/payment]
ytm_list = []
pv_list = []
spot_list = []
forward_list = []
forward_curve = []

ytm = float(Decimal(sp.irr(lst_)).quantize(Decimal('0.000000')))
print('YTM = ', str(ytm*100), '%')

# ytm
for i in range(1, nper+1):
    lst = [-current_bond_price] + [bond_par_value*rate/payment]*(i-1) + [bond_par_value+bond_par_value*rate/payment]
    ytm_list.append(float(Decimal(sp.irr(lst)).quantize(Decimal('0.000000'))))

# price
for i in range(nper):
    pv = -sp.pv(ytm, i+1, bond_par_value*rate/payment, bond_par_value)
    pv_list.append(float(Decimal(pv).quantize(Decimal('0.000000'))))
    
# spot rate
num = (bond_par_value*rate/payment) / (1 + ytm_list[0])
for i in range(nper):
    if i == 0 :
        spot_rate = ytm_list[i]
        spot_list.append(float(Decimal(spot_rate).quantize(Decimal('0.000000'))))
    else:
        spot_rate = ((bond_par_value + bond_par_value*rate/payment) / (current_bond_price - num)) ** (1/(i+1)) - 1
        spot_list.append(float(Decimal(spot_rate).quantize(Decimal('0.000000'))))
        num += (bond_par_value*rate/payment) / ((1 + spot_list[i]) ** (i+1))

# 輸出 ytm & spot rate & forward rate 表格與圖形
index_ = []
for i in range(nper):
    ytm_list[i] = ytm_list[i] * 100
    spot_list[i] = spot_list[i] * 100
    index_.append(str(str((i+1)*(1/2)) + '   '))
    
df1 = pd.DataFrame(ytm_list, columns = ['Ytm'], index = index_)
df2 = pd.DataFrame(spot_list, columns = ['Spot Rate'], index = index_)
df = pd.concat([df1,df2], axis = 1)
df.index.name = '到期時間(年)'
print(df)      

# 輸出 forward rate
pd.DataFrame(data = forward_list)

Current Bond Price: 820
Bond Par Value: 1000
Bond Coupon Rate(%): 6
Years to Maturity: 5
Payments a yaer: 2
YTM =  5.3736 %
             Ytm  Spot Rate
到期時間(年)                    
0.5      25.6098    25.6098
1.0      13.9200    13.7445
1.5      10.2729    10.0439
2.0       8.4945     8.2390
2.5       7.4421     7.1703
3.0       6.7469     6.4637
3.5       6.2535     5.9618
4.0       5.8855     5.5870
4.5       5.6006     5.2964
5.0       5.3736     5.0645


""


In [6]:
# forward rate

forward_list = []
for i in range(len(lst)):
    forward_list.append([])
    if i == 0:
        for j in range(len(lst)):
            if j == i:
                forward_list[i].append(0)
            else:
                forward_list[i].append(lst[j-1])
    else:
        for j in range(len(lst)):
            if j == i:
                forward_list[i].append(0)
            elif j > i:
                forward_rate = (((1 + lst[j-1]) ** (j) )/((1 + lst[i-1]) ** (i) )) **(1/(j-i)) - 1
                forward_rate = float(Decimal(forward_rate).quantize(Decimal('0.0000')))
                forward_list[i].append(forward_rate)
            else:
                forward_list[i].append(np.nan)

In [7]:
pd.DataFrame(data = forward_list)

,0,1,2,3,4,5,6,7,8,9
0,0.0,7.8947,5.1628,4.2676,3.8229,3.5569,3.3800,3.2538,3.1593,3.0858
1,NaN,0.0000,3.2700,3.0537,2.9328,2.8553,2.8014,2.7617,2.7313,2.7072
2,NaN,NaN,0.0000,2.8484,2.7743,2.7262,2.6925,2.6676,2.6484,2.6331
3,NaN,NaN,NaN,0.0000,2.7016,2.6665,2.6420,2.6237,2.6096,2.5984
4,NaN,NaN,NaN,NaN,0.0000,2.6317,2.6125,2.5981,2.5870,2.5781
5,NaN,NaN,NaN,NaN,NaN,0.0000,2.5933,2.5813,2.5722,2.5648
6,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,2.5694,2.5617,2.5554
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,2.5540,2.5484
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,2.5427
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
